In [1]:
# -*- coding: utf-8 -*-
"""
Huawei Network Automation Tool - Versão 3.1
Autor: Joseffer Maxwel 
Versão: 3.1 (2025) 
Requisitos: python3, ncclient, paramiko, tkinter
Observação: as configurações de conexão são mantidas em memória durante a execução.
"""
import os
import platform
import threading
import time
import subprocess
import sys
from datetime import datetime
from pathlib import Path
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox, filedialog

# bibliotecas de terceiros (instale via pip se necessário)
from ncclient import manager
import paramiko

# =========================
# CONFIGURAÇÕES INICIAIS 
# =========================
CONFIG_NETCONF = {
    "host": "192.168.56.100",
    "port": 830,
    "username": "netconf",
    "password": "Huawei12#$"
}

CONFIG_SSH = {
    "host": "192.168.56.100",
    "port": 22,
    "username": "python",
    "key_path": r"C:\Users\Joseffer\.ssh\id_rsa"
}

PASTA_BACKUPS_AUTO = os.path.join(str(Path.home()), "huawei_backups")

# Tema e fontes
TEMA = {
    "cor_accento": "#0B486B",
    "painel_lateral": "#263238",
    "btn_bg": "#37474F",
    "accent": "#00ACC1",
    "fundo": "#f5f5f5",
    "terminal_bg": "#1E1E1E",
    "terminal_fg": "#FFFFFF",
    "fonte_mono": "Consolas",
    "fonte_ui": "Segoe UI"
}

# =========================
# REGISTRO CENTRAL (MASTER LOG)
# =========================
_registro_central = []
_registro_lock = threading.Lock()
_visualizadores_realtime = [] 

def anexar_registro(texto: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    linha = f"[{ts}] {texto}"
    with _registro_lock:
        _registro_central.append(linha)
        if len(_registro_central) > 20000:
            _registro_central.pop(0)
    for viz in list(_visualizadores_realtime):
        try:
            fn = viz.get("append_fn")
            if fn and viz.get("enabled", True):
                fn(linha + "\n")
        except Exception:
            pass

def obter_texto_registro() -> str:
    with _registro_lock:
        return "\n".join(_registro_central)

def salvar_registro_em_arquivo(caminho: str) -> str:
    texto = obter_texto_registro()
    with open(caminho, "w", encoding="utf-8") as f:
        f.write(texto)
    return caminho

# =========================
# UTILITÁRIOS PARA WIDGETS (TEXT) E LOGS
# =========================
def configurar_tags(widget: scrolledtext.ScrolledText):
    try:
        widget.tag_config("INFO", foreground="#455A64")
        widget.tag_config("OK", foreground="#2E7D32", font=(TEMA["fonte_ui"], 9, "bold"))
        widget.tag_config("AVISO", foreground="#F57C00")
        widget.tag_config("ERRO", foreground="#C62828", font=(TEMA["fonte_ui"], 9, "bold"))
        widget.tag_config("HORA", foreground="#757575", font=(TEMA["fonte_ui"], 8, "italic"))
    except Exception:
        pass

def inserir_seguro(widget: scrolledtext.ScrolledText, texto: str, tags=None):
    try:
        widget.config(state="normal")
        if tags:
            widget.insert(tk.END, texto, tags)
        else:
            widget.insert(tk.END, texto)
        widget.see(tk.END)
        widget.config(state="disabled")
    except Exception:
        pass

def escrever_no_widget_log(widget: scrolledtext.ScrolledText, mensagem: str, nivel="INFO"):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        widget.config(state="normal")
        widget.insert(tk.END, f"[{timestamp}] ", ("HORA",))
        tag = nivel if nivel in ("OK", "ERRO", "AVISO") else "INFO"
        widget.insert(tk.END, f"[{nivel}] ", (tag,))
        widget.insert(tk.END, mensagem + "\n")
        widget.see(tk.END)
        widget.config(state="disabled")
    except Exception:
        pass
    anexar_registro(f"[{nivel}] {mensagem}")

def limpar_widget(widget: scrolledtext.ScrolledText):
    try:
        widget.config(state="normal")
        widget.delete("1.0", tk.END)
        widget.config(state="disabled")
    except Exception:
        pass

# =========================
# MODELO XML
# =========================
XML_MODELO = '''<config>
  <ethernet xmlns="http://www.huawei.com/netconf/vrp" content-version="1.0" format-version="1.0">
    <ethernetIfs>
{ethernet_block}
    </ethernetIfs>
  </ethernet>

  <ifm xmlns="http://www.huawei.com/netconf/vrp" content-version="1.0" format-version="1.0">
    <interfaces>
{interface_block}
    </interfaces>
  </ifm>
</config>'''

def gerar_xml_modelo() -> str:
    ethernet_block = ""
    interface_block = ""
    for i in range(1, 10):
        ethernet_block += f'''      <ethernetIf operation="merge">
        <ifName>GE1/0/{i}</ifName>
        <l2Enable>disable</l2Enable>
      </ethernetIf>\n'''
        interface_block += f'''      <interface operation="merge">
        <ifName>GE1/0/{i}</ifName>
        <ifDescr>Interface configurada via SCRIPT (GE1/0/{i})</ifDescr>
        <ifmAm4>
          <am4CfgAddrs>
            <am4CfgAddr operation="merge">
              <ifIpAddr>192.168.{i}.1</ifIpAddr>
              <subnetMask>255.255.255.0</subnetMask>
              <addrType>main</addrType>
            </am4CfgAddr>
          </am4CfgAddrs>
        </ifmAm4>
      </interface>\n'''
    return XML_MODELO.format(ethernet_block=ethernet_block.strip(), interface_block=interface_block.strip())

# =========================
# FUNÇÕES NETCONF
# =========================
def enviar_config_netconf_thread(xml_text: str, widget_log_ui):
    try:
        escrever_no_widget_log(widget_log_ui, f"🔗 Conectando a {CONFIG_NETCONF['host']}:{CONFIG_NETCONF['port']} (NETCONF)...")
        with manager.connect(
            host=CONFIG_NETCONF["host"],
            port=CONFIG_NETCONF["port"],
            username=CONFIG_NETCONF["username"],
            password=CONFIG_NETCONF["password"],
            hostkey_verify=False,
            device_params={"name": "huawei"},
            allow_agent=False,
            look_for_keys=False,
            timeout=15
        ) as con:
            escrever_no_widget_log(widget_log_ui, "✅ Sessão NETCONF estabelecida.")
            con.edit_config(target="running", config=xml_text)
            escrever_no_widget_log(widget_log_ui, "⚙️ Configuração aplicada com sucesso!", "OK")
    except Exception as e:
        escrever_no_widget_log(widget_log_ui, f"⚠️ Erro NETCONF: {e}", "ERRO")

def iniciar_envio_netconf(widget_log_ui, editor_xml_widget):
    xml_text = editor_xml_widget.get("1.0", tk.END).strip()
    if not xml_text:
        messagebox.showwarning("Aviso", "Insira ou importe um bloco XML antes de enviar.")
        return

    # 🔒 Verificação de campos obrigatórios (NETCONF)
    if not CONFIG_NETCONF["host"] or not CONFIG_NETCONF["username"] or not CONFIG_NETCONF["password"] or not CONFIG_NETCONF["port"]:
        messagebox.showerror(
            "Erro de Configuração",
            "Os parâmetros de conexão NETCONF estão incompletos.\n\n"
            "Verifique o Host/IP, Porta, Usuário e Senha na aba ⚙️ Conexões antes de continuar."
        )
        escrever_no_widget_log(widget_log_ui, "❌ Envio bloqueado: parâmetros NETCONF incompletos.", "ERRO")
        return

    confirmar = messagebox.askyesno("Confirmar Envio", "Deseja realmente aplicar esta configuração no equipamento?")
    if not confirmar:
        escrever_no_widget_log(widget_log_ui, "❌ Envio cancelado pelo usuário.", "AVISO")
        return

    # Tudo validado — iniciar envio
    t = threading.Thread(target=enviar_config_netconf_thread, args=(xml_text, widget_log_ui), daemon=True)
    t.start()

# =========================
# FUNÇÕES DE BACKUP (SSH)
# =========================
def extrair_config_ssh(widget_log_ui, widget_saida_backup, timeout_seg=25):
    limpar_widget(widget_saida_backup)
    try:
        escrever_no_widget_log(widget_log_ui, f"🔗 Conectando a {CONFIG_SSH['host']} via SSH para extração...")
        cliente = paramiko.SSHClient()
        cliente.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        cliente.connect(
            hostname=CONFIG_SSH["host"],
            port=CONFIG_SSH["port"],
            username=CONFIG_SSH["username"],
            key_filename=CONFIG_SSH["key_path"],
            timeout=15
        )
        canal = cliente.invoke_shell()
        canal.send('screen-length 0 temporary\n')
        canal.send('display current-configuration\n')
        saida = ""
        inicio = time.time()
        while time.time() - inicio < timeout_seg:
            if canal.recv_ready():
                parte = canal.recv(4096).decode(errors='ignore')
                saida += parte
                if "# " in parte or "return" in parte.lower():
                    break
            time.sleep(0.2)
        canal.close()
        cliente.close()
        inserir_seguro(widget_saida_backup, saida)
        escrever_no_widget_log(widget_log_ui, "💾 Extração concluída e exibida.", "OK")
    except Exception as e:
        escrever_no_widget_log(widget_log_ui, f"⚠️ Erro ao executar backup SSH: {e}", "ERRO")

def salvar_conteudo_texto(texto: str, titulo="Salvar arquivo", def_ext=".txt", tipos=(("TXT","*.txt"),)):
    caminho = filedialog.asksaveasfilename(title=titulo, defaultextension=def_ext, filetypes=tipos)
    if caminho:
        with open(caminho, "w", encoding="utf-8") as f:
            f.write(texto)
        return caminho
    return None

# BACKUP AUTOMÁTICO
_thread_backup_auto = None
_backup_auto_ativado = False

def loop_backup_automatico(intervalo_min, widget_log_ui, pasta_salvar):
    global _backup_auto_ativado
    while _backup_auto_ativado:
        try:
            escrever_no_widget_log(widget_log_ui, f"🕒 Executando backup automático ({datetime.now().strftime('%H:%M:%S')})...")
            cliente = paramiko.SSHClient()
            cliente.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            cliente.connect(
                hostname=CONFIG_SSH["host"],
                port=CONFIG_SSH["port"],
                username=CONFIG_SSH["username"],
                key_filename=CONFIG_SSH["key_path"],
                timeout=15
            )
            canal = cliente.invoke_shell()
            canal.send('screen-length 0 temporary\n')
            canal.send('display current-configuration\n')
            saida = ""
            inicio = time.time()
            while time.time() - inicio < 25:
                if canal.recv_ready():
                    parte = canal.recv(4096).decode(errors='ignore')
                    saida += parte
                    if "return" in parte.lower() or "# " in parte:
                        break
                time.sleep(0.2)
            canal.close()
            cliente.close()
            os.makedirs(pasta_salvar, exist_ok=True)
            nome = f"backup_auto_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt"
            caminho = os.path.join(pasta_salvar, nome)
            with open(caminho, "w", encoding="utf-8") as f:
                f.write(saida)
            escrever_no_widget_log(widget_log_ui, f"💾 Backup automático salvo: {caminho}", "OK")
        except Exception as e:
            escrever_no_widget_log(widget_log_ui, f"⚠️ Erro durante backup automático: {e}", "ERRO")
        for _ in range(max(1, int(intervalo_min * 60))):
            if not _backup_auto_ativado:
                break
            time.sleep(1)

def iniciar_backup_automatico(intervalo_min, widget_log_ui, pasta_salvar):
    global _thread_backup_auto, _backup_auto_ativado
    if _backup_auto_ativado:
        escrever_no_widget_log(widget_log_ui, "⚠️ Backup automático já está em execução.", "AVISO")
        return
    _backup_auto_ativado = True
    _thread_backup_auto = threading.Thread(target=loop_backup_automatico, args=(intervalo_min, widget_log_ui, pasta_salvar), daemon=True)
    _thread_backup_auto.start()
    escrever_no_widget_log(widget_log_ui, f"▶️ Backup automático iniciado a cada {intervalo_min} min — Pasta: {pasta_salvar}", "OK")

def parar_backup_automatico(widget_log_ui):
    global _backup_auto_ativado
    if _backup_auto_ativado:
        _backup_auto_ativado = False
        escrever_no_widget_log(widget_log_ui, "⏹️ Pedido de parada enviado para backup automático.", "INFO")
    else:
        escrever_no_widget_log(widget_log_ui, "ℹ️ Nenhum backup automático em execução.", "INFO")

# =========================
# CONSOLE SSH INTERATIVO 
# =========================
sessao_ssh = None
canal_ssh = None
ssh_conectado = False

def terminal_escrever(texto: str):
    try:
        log_console.config(state="normal")
        log_console.insert(tk.END, texto)
        log_console.see(tk.END)
        log_console.config(state="disabled")
    except Exception:
        pass

def conectar_ssh_terminal():
    global sessao_ssh, canal_ssh, ssh_conectado
    terminal_escrever(f"🔗 Conectando ao dispositivo {CONFIG_SSH['host']}:{CONFIG_SSH['port']}...\n")
    try:
        sessao_ssh = paramiko.SSHClient()
        sessao_ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        sessao_ssh.connect(
            hostname=CONFIG_SSH["host"],
            port=CONFIG_SSH["port"],
            username=CONFIG_SSH["username"],
            key_filename=CONFIG_SSH["key_path"],
            timeout=10
        )
        canal_ssh = sessao_ssh.invoke_shell()
        canal_ssh.settimeout(0.1)
        ssh_conectado = True
        terminal_escrever("✅ Sessão SSH interativa estabelecida!\n\n")
        anexar_registro("[SSH] Sessão interativa estabelecida.")
        threading.Thread(target=receber_saida_ssh, daemon=True).start()
        log_console.config(state="normal")
        log_console.mark_set(tk.INSERT, tk.END)
        log_console.focus()
    except Exception as e:
        terminal_escrever(f"⚠️ Erro ao conectar: {e}\n")
        anexar_registro(f"[SSH] Erro ao conectar: {e}")
        ssh_conectado = False

def desconectar_ssh_terminal():
    global sessao_ssh, canal_ssh, ssh_conectado
    try:
        ssh_conectado = False
        if canal_ssh:
            canal_ssh.close()
        if sessao_ssh:
            sessao_ssh.close()
        log_console.config(state="normal")
        log_console.delete("1.0", tk.END)
        log_console.insert(tk.END, "📴 Conexão encerrada.\n\n⚠️ Nenhum usuário conectado ao equipamento.\n")
        log_console.config(state="disabled")
        anexar_registro("[SSH] Sessão encerrada pelo usuário.")
    except Exception as e:
        log_console.config(state="normal")
        log_console.insert(tk.END, f"⚠️ Erro ao desconectar: {e}\n")
        log_console.config(state="disabled")
        anexar_registro(f"[SSH] Erro ao desconectar: {e}")

def receber_saida_ssh():
    global ssh_conectado, canal_ssh
    try:
        while ssh_conectado and canal_ssh:
            if canal_ssh.recv_ready():
                dados = canal_ssh.recv(4096).decode(errors='ignore')
                terminal_escrever(dados)
            time.sleep(0.05)
    except Exception:
        pass

def teclado_terminal(event):
    global canal_ssh, ssh_conectado
    if not ssh_conectado or not canal_ssh:
        return "break"
    try:
        tecla = event.char
        if tecla:
            canal_ssh.send(tecla)
    except Exception as e:
        terminal_escrever(f"\n❌ Erro ao enviar tecla: {e}\n")
        anexar_registro(f"[SSH] Erro ao enviar tecla: {e}")
    return "break"

# =========================
# TESTES DE CONECTIVIDADE (ping / traceroute)
# =========================
def executar_processo_para_widget(cmd_lista, widget_log_ui):
    try:
        proc = subprocess.Popen(cmd_lista, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for linha in proc.stdout:
            escrever_no_widget_log(widget_log_ui, linha.rstrip())
        proc.wait()
        escrever_no_widget_log(widget_log_ui, "✅ Teste concluído.", "OK")
    except Exception as e:
        escrever_no_widget_log(widget_log_ui, f"❌ Erro: {e}", "ERRO")

def executar_ping(widget_log_ui, alvo):
    if not alvo:
        messagebox.showwarning("Aviso", "Insira um IP válido.")
        return
    limpar_widget(widget_log_ui)
    escrever_no_widget_log(widget_log_ui, f"📡 Executando ping para {alvo}...")
    cmd = ["ping", "-n", "4", alvo] if platform.system().lower() == "windows" else ["ping", "-c", "4", alvo]
    threading.Thread(target=executar_processo_para_widget, args=(cmd, widget_log_ui), daemon=True).start()

def executar_traceroute(widget_log_ui, alvo):
    if not alvo:
        messagebox.showwarning("Aviso", "Insira um IP válido.")
        return
    limpar_widget(widget_log_ui)
    escrever_no_widget_log(widget_log_ui, f"🛰️ Executando traceroute para {alvo}...")
    cmd = ["tracert", alvo] if platform.system().lower() == "windows" else ["traceroute", alvo]
    threading.Thread(target=executar_processo_para_widget, args=(cmd, widget_log_ui), daemon=True).start()

# =========================
# MONTAGEM DA INTERFACE (UI)
# =========================
root = tk.Tk()
root.title("Huawei Network Automation Tool - v3.1 (Português)")
root.geometry("1100x750")
root.configure(bg=TEMA["fundo"])

# Painel lateral esquerdo
painel_esquerdo = tk.Frame(root, width=240, bg=TEMA["painel_lateral"])
painel_esquerdo.pack(side="left", fill="y")

tk.Label(painel_esquerdo, text="HUAWEI TOOL", bg=TEMA["painel_lateral"], fg="white",
         font=(TEMA["fonte_ui"], 14, "bold")).pack(pady=18)

container_botoes = tk.Frame(painel_esquerdo, bg=TEMA["painel_lateral"])
container_botoes.pack(fill="both", expand=True)

LARG_BOTAO = 22
PAD_Y = 8

# Área direita (conteúdo)
area_direita = tk.Frame(root, bg=TEMA["fundo"])
area_direita.pack(side="right", fill="both", expand=True)

def centralizar_conteudo(frame):
    painel = tk.Frame(frame, bg=TEMA["fundo"])
    painel.pack(expand=True)
    return painel

# Frames principais
frame_netconf = tk.Frame(area_direita, bg=TEMA["fundo"])
frame_backup = tk.Frame(area_direita, bg=TEMA["fundo"])
frame_teste = tk.Frame(area_direita, bg=TEMA["fundo"])
frame_console = tk.Frame(area_direita, bg=TEMA["fundo"])
frame_logs = tk.Frame(area_direita, bg=TEMA["fundo"])
frame_config = tk.Frame(area_direita, bg=TEMA["fundo"])  

TODOS_FRAMES = (frame_netconf, frame_backup, frame_teste, frame_console, frame_logs, frame_config)

def mostrar_frame(qual):
    for f in TODOS_FRAMES:
        try:
            f.pack_forget()
        except Exception:
            pass
    qual.pack(fill="both", expand=True)

# Botões laterais
tk.Button(container_botoes, text="⚙️ Aplicar Configuração", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_netconf)).pack(pady=PAD_Y)

tk.Button(container_botoes, text="🧠 Backup e Extração", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_backup)).pack(pady=PAD_Y)

tk.Button(container_botoes, text="🔎 Testar Conectividade", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_teste)).pack(pady=PAD_Y)

tk.Button(container_botoes, text="🖥️ Console SSH", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_console)).pack(pady=PAD_Y)

tk.Button(container_botoes, text="⚙️ Conexões", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_config)).pack(pady=PAD_Y)

tk.Button(container_botoes, text="📘 Logs do Sistema", bg=TEMA["btn_bg"], fg="white",
          font=(TEMA["fonte_ui"], 10, "bold"), relief="flat", width=LARG_BOTAO,
          command=lambda: mostrar_frame(frame_logs)).pack(pady=PAD_Y)

# Sobre no rodapé
frame_sobre = tk.Frame(painel_esquerdo, bg=TEMA["painel_lateral"])
frame_sobre.pack(side="bottom", fill="x", pady=14)

def abrir_sobre():
    j = tk.Toplevel(root)
    j.title("Sobre o Projeto")
    j.geometry("600x400")
    j.configure(bg=TEMA["fundo"])
    tk.Label(j, text="Sobre o Projeto", font=(TEMA["fonte_ui"], 14, "bold"), bg=TEMA["fundo"]).pack(pady=12)
    texto = (
    "Este software foi desenvolvido por Joseffer Maxwel como parte do Trabalho de Conclusão de Curso "
    "em Tecnologia em Telemática – IFPB Campus Campina Grande.\n\n"
    "A ferramenta tem caráter exclusivamente acadêmico e demonstra a aplicação prática da automação de redes "
    "utilizando Python, ncclient e Paramiko, simulando a comunicação com equipamentos compatíveis com os protocolos "
    "SSHv2 e NETCONF.\n\n"
    "Este projeto não possui qualquer vínculo, parceria, autorização ou associação oficial com a Huawei Technologies Co., Ltd. "
    "O nome 'Huawei' é utilizado apenas para fins educacionais e de identificação técnica dentro do contexto da automação de redes.\n\n"
    "Versão 3.1."
    )
    tk.Label(j, text=texto, wraplength=560, justify="left", bg=TEMA["fundo"], font=(TEMA["fonte_ui"], 10)).pack(padx=20, pady=10)
    tk.Button(j, text="Fechar", bg="#4CAF50", fg="white", command=j.destroy).pack(pady=8)

tk.Button(frame_sobre, text="ℹ️ Sobre o Projeto", bg="#546E7A", fg="white", font=(TEMA["fonte_ui"], 10, "bold"),
          relief="flat", width=LARG_BOTAO, command=abrir_sobre).pack()

# ---------------------------
# Frame NETCONF
# ---------------------------
c_netconf = centralizar_conteudo(frame_netconf)
tk.Label(c_netconf, text="Bloco XML de configuração (modelo base):", font=(TEMA["fonte_ui"], 11, "bold"), bg=TEMA["fundo"]).pack(pady=6)
editor_xml = scrolledtext.ScrolledText(c_netconf, width=96, height=18, font=(TEMA["fonte_mono"], 10))
editor_xml.pack(padx=10, pady=5)
editor_xml.insert(tk.END, gerar_xml_modelo())

botoes_netconf = tk.Frame(c_netconf, bg=TEMA["fundo"])
botoes_netconf.pack(pady=6)

log_netconf = scrolledtext.ScrolledText(c_netconf, width=96, height=7, font=(TEMA["fonte_mono"], 10))
log_netconf.pack(padx=10, pady=6)
configurar_tags(log_netconf)

def importar_xml_editor():
    caminho = filedialog.askopenfilename(title="Selecionar arquivo XML", filetypes=(("XML/TXT","*.xml *.txt"),("Todos","*.*")))
    if caminho:
        try:
            with open(caminho, "r", encoding="utf-8") as f:
                conteudo = f.read()
            editor_xml.delete("1.0", tk.END)
            editor_xml.insert(tk.END, conteudo)
            escrever_no_widget_log(log_netconf, f"📂 Arquivo importado: {os.path.basename(caminho)}")
        except Exception as e:
            escrever_no_widget_log(log_netconf, f"⚠️ Erro ao importar XML: {e}", "ERRO")

def exportar_xml_editor():
    conteudo = editor_xml.get("1.0", tk.END).strip()
    if not conteudo:
        messagebox.showwarning("Aviso", "Nada para exportar.")
        return
    caminho = filedialog.asksaveasfilename(title="Exportar XML", defaultextension=".xml", filetypes=(("XML","*.xml"),("TXT","*.txt")))
    if caminho:
        try:
            with open(caminho, "w", encoding="utf-8") as f:
                f.write(conteudo)
            escrever_no_widget_log(log_netconf, f"💾 XML exportado: {os.path.basename(caminho)}", "OK")
        except Exception as e:
            escrever_no_widget_log(log_netconf, f"⚠️ Erro ao exportar XML: {e}", "ERRO")

tk.Button(botoes_netconf, text="🚀 Enviar", bg="#4CAF50", fg="white", width=10,
          command=lambda: iniciar_envio_netconf(log_netconf, editor_xml)).grid(row=0, column=0, padx=6)
tk.Button(botoes_netconf, text="📂 Importar", bg="#2196F3", fg="white", width=10,
          command=importar_xml_editor).grid(row=0, column=1, padx=6)
tk.Button(botoes_netconf, text="💾 Exportar", bg="#9C27B0", fg="white", width=10,
          command=exportar_xml_editor).grid(row=0, column=2, padx=6)

# ---------------------------
# Frame BACKUP
# ---------------------------
c_backup = centralizar_conteudo(frame_backup)
tk.Label(c_backup, text="Visualização da configuração atual do equipamento:", font=(TEMA["fonte_ui"], 11, "bold"), bg=TEMA["fundo"]).pack(pady=6)
saida_backup = scrolledtext.ScrolledText(c_backup, width=96, height=14, font=(TEMA["fonte_mono"], 10))
saida_backup.pack(padx=10, pady=5)
saida_backup.config(state="disabled")

botoes_backup = tk.Frame(c_backup, bg=TEMA["fundo"])
botoes_backup.pack(pady=6)
tk.Button(botoes_backup, text="📥 Ver Configuração", bg="#4CAF50", fg="white", width=16,
          command=lambda: threading.Thread(target=extrair_config_ssh, args=(log_ssh, saida_backup), daemon=True).start()).grid(row=0, column=0, padx=6)
tk.Button(botoes_backup, text="💾 Salvar Backup", bg="#2196F3", fg="white", width=16,
          command=lambda: salvar_backup_atual(saida_backup, log_ssh)).grid(row=0, column=1, padx=6)

def salvar_backup_atual(widget_saida, widget_log_ui):
    conteudo = widget_saida.get("1.0", tk.END).strip()
    if not conteudo:
        messagebox.showwarning("Aviso", "Nada para salvar no backup.")
        return
    caminho = salvar_conteudo_texto(conteudo, titulo="Salvar Backup", def_ext=".txt")
    if caminho:
        escrever_no_widget_log(widget_log_ui, f"💾 Backup manual salvo: {os.path.basename(caminho)}", "OK")

# Auto backup
frame_auto = tk.Frame(c_backup, bg=TEMA["fundo"])
frame_auto.pack(pady=6)
tk.Label(frame_auto, text="Pasta backups automáticos:", bg=TEMA["fundo"]).grid(row=0, column=0, sticky="w")
entrada_pasta_auto = tk.Entry(frame_auto, width=60)
entrada_pasta_auto.grid(row=0, column=1, padx=6)
entrada_pasta_auto.insert(0, PASTA_BACKUPS_AUTO)

def escolher_pasta_auto():
    p = filedialog.askdirectory(title="Selecionar pasta para backups automáticos")
    if p:
        entrada_pasta_auto.delete(0, tk.END)
        entrada_pasta_auto.insert(0, p)

tk.Button(frame_auto, text="📁", width=3, command=escolher_pasta_auto).grid(row=0, column=2, padx=4)
tk.Label(frame_auto, text="Intervalo (min):", bg=TEMA["fundo"]).grid(row=1, column=0, sticky="w", pady=6)
entrada_intervalo = tk.Entry(frame_auto, width=10)
entrada_intervalo.grid(row=1, column=1, sticky="w")
entrada_intervalo.insert(0, "60")

btn_iniciar_auto = tk.Button(frame_auto, text="▶️ Iniciar Backup Automático", bg="#4CAF50", fg="white", width=22)
btn_parar_auto = tk.Button(frame_auto, text="⏹️ Parar Backup Automático", bg="#F44336", fg="white", width=22)
btn_iniciar_auto.grid(row=1, column=1, sticky="e", padx=8)
btn_parar_auto.grid(row=1, column=2, sticky="w", padx=4)

def on_iniciar_auto():
    try:
        intervalo = int(entrada_intervalo.get())
        pasta = entrada_pasta_auto.get().strip() or PASTA_BACKUPS_AUTO
        iniciar_backup_automatico(intervalo, log_ssh, pasta)
    except Exception as e:
        messagebox.showerror("Erro", f"Intervalo inválido: {e}")

def on_parar_auto():
    parar_backup_automatico(log_ssh)

btn_iniciar_auto.config(command=on_iniciar_auto)
btn_parar_auto.config(command=on_parar_auto)

log_ssh = scrolledtext.ScrolledText(c_backup, width=96, height=7, font=(TEMA["fonte_mono"], 10))
log_ssh.pack(padx=10, pady=6)
configurar_tags(log_ssh)

# ---------------------------
# Frame TESTES (Ping / Traceroute)
# ---------------------------
c_teste = centralizar_conteudo(frame_teste)
tk.Label(c_teste, text="Teste de Conectividade (Ping / Traceroute)", font=(TEMA["fonte_ui"], 11, "bold"), bg=TEMA["fundo"]).pack(pady=10)
frame_ip = tk.Frame(c_teste, bg=TEMA["fundo"])
frame_ip.pack(pady=6)
tk.Label(frame_ip, text="Endereço IP ou Host:", bg=TEMA["fundo"]).grid(row=0, column=0, padx=4)
entrada_ip = tk.Entry(frame_ip, width=30, font=(TEMA["fonte_ui"], 10))
entrada_ip.grid(row=0, column=1, padx=4)
frame_btns_teste = tk.Frame(c_teste, bg=TEMA["fundo"])
frame_btns_teste.pack(pady=8)
tk.Button(frame_btns_teste, text="📡 Ping", bg="#4CAF50", fg="white", width=12,
          command=lambda: executar_ping(log_teste, entrada_ip.get().strip())).grid(row=0, column=0, padx=8)
tk.Button(frame_btns_teste, text="🛰 Traceroute", bg="#2196F3", fg="white", width=12,
          command=lambda: executar_traceroute(log_teste, entrada_ip.get().strip())).grid(row=0, column=1, padx=8)
log_teste = scrolledtext.ScrolledText(c_teste, width=96, height=12, font=(TEMA["fonte_mono"], 10))
log_teste.pack(padx=10, pady=5)
configurar_tags(log_teste)

# ---------------------------
# Frame CONSOLE SSH
# ---------------------------
c_console = centralizar_conteudo(frame_console)
tk.Label(c_console, text="Console SSH Interativo", font=(TEMA["fonte_ui"], 11, "bold"), bg=TEMA["fundo"]).pack(pady=8)

log_console = scrolledtext.ScrolledText(c_console, width=96, height=22, font=(TEMA["fonte_mono"], 10),
                                       bg=TEMA["terminal_bg"], fg=TEMA["terminal_fg"], insertbackground=TEMA["terminal_fg"])
log_console.pack(padx=10, pady=8)
log_console.insert(tk.END, "🔘 Aguardando conexão...\n\n")
log_console.config(state="disabled")

log_console.bind("<Key>", teclado_terminal)

frame_cmd = tk.Frame(c_console, bg=TEMA["fundo"])
frame_cmd.pack(pady=10)
tk.Button(frame_cmd, text="🔌 Conectar", bg="#4CAF50", fg="white",
          width=12, command=lambda: threading.Thread(target=conectar_ssh_terminal, daemon=True).start()).grid(row=0, column=0, padx=8)
tk.Button(frame_cmd, text="❌ Desconectar", bg="#F44336", fg="white",
          width=14, command=lambda: threading.Thread(target=desconectar_ssh_terminal, daemon=True).start()).grid(row=0, column=1, padx=8)

# ---------------------------
# Frame LOGS DO SISTEMA (registro central)
# ---------------------------
c_logs = centralizar_conteudo(frame_logs)
tk.Label(c_logs, text="Logs do Sistema (consolidados)", font=(TEMA["fonte_ui"], 11, "bold"), bg=TEMA["fundo"]).pack(pady=6)
logs_system_text = scrolledtext.ScrolledText(c_logs, width=96, height=30, font=(TEMA["fonte_mono"], 10))
logs_system_text.pack(padx=10, pady=5)
logs_system_text.config(state="normal")
logs_system_text.insert(tk.END, obter_texto_registro() + "\n")
logs_system_text.config(state="disabled")

def logs_append_fn(linha):
    try:
        logs_system_text.config(state="normal")
        logs_system_text.insert(tk.END, linha)
        logs_system_text.see(tk.END)
        logs_system_text.config(state="disabled")
    except Exception:
        pass

viz_info = {"append_fn": logs_append_fn, "enabled": True}
_visualizadores_realtime.append(viz_info)

ctl_logs = tk.Frame(c_logs, bg=TEMA["fundo"])
ctl_logs.pack(fill="x", padx=10, pady=(0,8))
tk.Button(ctl_logs, text="💾 Salvar Logs", bg="#2196F3", fg="white",
          command=lambda: salvar_registro_ui()).pack(side="left", padx=6)

def salvar_registro_ui():
    caminho = filedialog.asksaveasfilename(title="Salvar Log Consolidado", defaultextension=".log", filetypes=(("LOG","*.log"),("TXT","*.txt")))
    if caminho:
        salvar_registro_em_arquivo(caminho)
        messagebox.showinfo("Salvo", f"Log salvo em: {caminho}")

# ---------------------------
# Frame CONFIGURAÇÕES 
# ---------------------------
c_config = centralizar_conteudo(frame_config)
tk.Label(c_config, text="Configurações de Dispositivo (em memória)", font=(TEMA["fonte_ui"], 12, "bold"), bg=TEMA["fundo"]).pack(pady=8)

frame_cfg_netconf = tk.LabelFrame(c_config, text="NETCONF", bg=TEMA["fundo"], padx=8, pady=8)
frame_cfg_netconf.pack(fill="x", padx=12, pady=(6,10))

tk.Label(frame_cfg_netconf, text="Host / IP:", bg=TEMA["fundo"]).grid(row=0, column=0, sticky="e", padx=6, pady=4)
ent_netconf_host = tk.Entry(frame_cfg_netconf, width=36)
ent_netconf_host.grid(row=0, column=1, padx=6, pady=4)
ent_netconf_host.insert(0, CONFIG_NETCONF["host"])

tk.Label(frame_cfg_netconf, text="Porta NETCONF:", bg=TEMA["fundo"]).grid(row=1, column=0, sticky="e", padx=6, pady=4)
ent_netconf_port = tk.Entry(frame_cfg_netconf, width=12)
ent_netconf_port.grid(row=1, column=1, sticky="w", padx=6, pady=4)
ent_netconf_port.insert(0, str(CONFIG_NETCONF["port"]))

tk.Label(frame_cfg_netconf, text="Usuário NETCONF:", bg=TEMA["fundo"]).grid(row=2, column=0, sticky="e", padx=6, pady=4)
ent_netconf_user = tk.Entry(frame_cfg_netconf, width=36)
ent_netconf_user.grid(row=2, column=1, padx=6, pady=4)
ent_netconf_user.insert(0, CONFIG_NETCONF["username"])

tk.Label(frame_cfg_netconf, text="Senha NETCONF:", bg=TEMA["fundo"]).grid(row=3, column=0, sticky="e", padx=6, pady=4)
ent_netconf_pwd = tk.Entry(frame_cfg_netconf, width=36, show="*")
ent_netconf_pwd.grid(row=3, column=1, padx=6, pady=4)
ent_netconf_pwd.insert(0, CONFIG_NETCONF["password"])

frame_cfg_ssh = tk.LabelFrame(c_config, text="SSH", bg=TEMA["fundo"], padx=8, pady=8)
frame_cfg_ssh.pack(fill="x", padx=12, pady=(0,10))

tk.Label(frame_cfg_ssh, text="Host / IP:", bg=TEMA["fundo"]).grid(row=0, column=0, sticky="e", padx=6, pady=4)
ent_ssh_host = tk.Entry(frame_cfg_ssh, width=36)
ent_ssh_host.grid(row=0, column=1, padx=6, pady=4)
ent_ssh_host.insert(0, CONFIG_SSH["host"])

tk.Label(frame_cfg_ssh, text="Porta SSH:", bg=TEMA["fundo"]).grid(row=1, column=0, sticky="e", padx=6, pady=4)
ent_ssh_port = tk.Entry(frame_cfg_ssh, width=12)
ent_ssh_port.grid(row=1, column=1, sticky="w", padx=6, pady=4)
ent_ssh_port.insert(0, str(CONFIG_SSH["port"]))

tk.Label(frame_cfg_ssh, text="Usuário SSH:", bg=TEMA["fundo"]).grid(row=2, column=0, sticky="e", padx=6, pady=4)
ent_ssh_user = tk.Entry(frame_cfg_ssh, width=36)
ent_ssh_user.grid(row=2, column=1, padx=6, pady=4)
ent_ssh_user.insert(0, CONFIG_SSH["username"])

tk.Label(frame_cfg_ssh, text="Caminho da chave (key):", bg=TEMA["fundo"]).grid(row=3, column=0, sticky="e", padx=6, pady=4)
ent_ssh_key = tk.Entry(frame_cfg_ssh, width=36)
ent_ssh_key.grid(row=3, column=1, padx=6, pady=4)
ent_ssh_key.insert(0, CONFIG_SSH["key_path"])

def selecionar_chave_ssh():
    p = filedialog.askopenfilename(title="Selecionar chave privada (key)", filetypes=(("All files","*.*"),))
    if p:
        ent_ssh_key.delete(0, tk.END)
        ent_ssh_key.insert(0, p)

tk.Button(frame_cfg_ssh, text="📁", width=3, command=selecionar_chave_ssh).grid(row=3, column=2, padx=6)

# Botões de salvar/atualizar (em memória)
frame_cfg_actions = tk.Frame(c_config, bg=TEMA["fundo"])
frame_cfg_actions.pack(pady=8)

def aplicar_configuracoes_em_memoria():
    """Aplica configurações e valida conexões silenciosamente (com aviso de atualização)."""
    def validar_conexoes_rapido():
        global conexoes_validadas
        conexoes_validadas = {"netconf": False, "ssh": False}
        resultados = []

        anexar_registro("[CONFIG] Iniciando validação rápida de conexões...")

        def validar_netconf():
            try:
                anexar_registro("[CONFIG] Validando conexão NETCONF...")
                with manager.connect(
                    host=CONFIG_NETCONF["host"],
                    port=CONFIG_NETCONF["port"],
                    username=CONFIG_NETCONF["username"],
                    password=CONFIG_NETCONF["password"],
                    hostkey_verify=False,
                    device_params={"name": "huawei"},
                    allow_agent=False,
                    look_for_keys=False,
                    timeout=6
                ) as m:
                    try:
                        _ = m.server_capabilities
                    except Exception:
                        pass
                    conexoes_validadas["netconf"] = True
                    resultados.append("✅ NETCONF: sucesso")
                    anexar_registro("[CONFIG] ✅ NETCONF validado com sucesso.")
            except Exception as e:
                if "close" in str(e).lower() or "session" in str(e).lower():
                    conexoes_validadas["netconf"] = True
                    resultados.append("✅ NETCONF: sucesso (sessão encerrada após validação)")
                    anexar_registro("[CONFIG] ⚠️ NETCONF validado, mas sessão fechada logo após handshake.")
                else:
                    resultados.append(f"❌ NETCONF: {e}")
                    anexar_registro(f"[CONFIG] ❌ Falha na validação NETCONF: {e}")

        def validar_ssh():
            try:
                anexar_registro("[CONFIG] Validando conexão SSH...")
                cliente = paramiko.SSHClient()
                cliente.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                cliente.connect(
                    hostname=CONFIG_SSH["host"],
                    port=CONFIG_SSH["port"],
                    username=CONFIG_SSH["username"],
                    key_filename=CONFIG_SSH["key_path"],
                    timeout=3
                )
                canal = cliente.invoke_shell()
                time.sleep(0.2)
                if canal.active:
                    conexoes_validadas["ssh"] = True
                    resultados.append("✅ SSH: sucesso")
                    anexar_registro("[CONFIG] ✅ SSH validado com sucesso.")
                else:
                    resultados.append("❌ SSH: canal não ativo.")
                    anexar_registro("[CONFIG] ❌ SSH canal não ativo.")
                canal.close()
                cliente.close()
            except Exception as e:
                resultados.append(f"❌ SSH: {e}")
                anexar_registro(f"[CONFIG] ❌ Falha na validação SSH: {e}")

        # Executa ambas em paralelo
        t1 = threading.Thread(target=validar_netconf, daemon=True)
        t2 = threading.Thread(target=validar_ssh, daemon=True)
        t1.start()
        t2.start()

        # Espera finalização breve
        for _ in range(10):
            if not (t1.is_alive() or t2.is_alive()):
                break
            time.sleep(0.2)
            root.update_idletasks()

        t1.join(timeout=0.1)
        t2.join(timeout=0.1)

        # Apenas log final
        msg = "\n".join(resultados)
        if conexoes_validadas["netconf"] and conexoes_validadas["ssh"]:
            anexar_registro("[CONFIG] ✅ Todas as conexões validadas com sucesso.")
        else:
            anexar_registro(f"[CONFIG] ⚠️ Resultado da validação:\n{msg}")

    # --- Atualiza parâmetros em memória ---
    try:
        CONFIG_NETCONF["host"] = ent_netconf_host.get().strip() or CONFIG_NETCONF["host"]
        CONFIG_NETCONF["port"] = int(ent_netconf_port.get().strip() or CONFIG_NETCONF["port"])
        CONFIG_NETCONF["username"] = ent_netconf_user.get().strip() or CONFIG_NETCONF["username"]
        CONFIG_NETCONF["password"] = ent_netconf_pwd.get() or CONFIG_NETCONF["password"]

        CONFIG_SSH["host"] = ent_ssh_host.get().strip() or CONFIG_SSH["host"]
        CONFIG_SSH["port"] = int(ent_ssh_port.get().strip() or CONFIG_SSH["port"])
        CONFIG_SSH["username"] = ent_ssh_user.get().strip() or CONFIG_SSH["username"]
        CONFIG_SSH["key_path"] = ent_ssh_key.get().strip() or CONFIG_SSH["key_path"]

        anexar_registro("[CONFIG] Configurações aplicadas em memória. Iniciando validação silenciosa...")
        messagebox.showinfo("Configurações", "✅ Conexões atualizadas!\n\nA validação será executada em segundo plano.")
        threading.Thread(target=validar_conexoes_rapido, daemon=True).start()

    except Exception as e:
        anexar_registro(f"[CONFIG] ❌ Erro ao aplicar configurações: {e}")

# Botões de salvar/atualizar (em memória)
frame_cfg_actions = tk.Frame(c_config, bg=TEMA["fundo"])
frame_cfg_actions.pack(pady=8)

btn_aplicar = tk.Button(
    frame_cfg_actions,
    text="💾 Aplicar Configurações (em memória)",
    bg="#2196F3",
    fg="white",
    font=(TEMA["fonte_ui"], 10, "bold"),
    width=35,
    command=aplicar_configuracoes_em_memoria
)
btn_aplicar.pack(padx=6, pady=4)

# ---------------------------
# Finalizações e comportamento da janela
# ---------------------------
mostrar_frame(frame_netconf)
anexar_registro("[SYSTEM] Aplicação inicializada (v3.1 - configurações em memória).")

def ao_fechar():
    if messagebox.askokcancel("Sair", "Deseja realmente encerrar a aplicação?"):
        try:
            parar_backup_automatico(log_ssh)
        except Exception:
            pass
        try:
            if sessao_ssh:
                sessao_ssh.close()
        except Exception:
            pass
        try:
            _visualizadores_realtime.remove(viz_info)
        except Exception:
            pass
        anexar_registro("[SYSTEM] Aplicação encerrada pelo usuário.")
        root.destroy()

root.protocol("WM_DELETE_WINDOW", ao_fechar)

# foco inicial no console
root.after(300, lambda: (log_console.focus_set(), log_console.mark_set(tk.INSERT, tk.END)))

# executar loop principal
if __name__ == "__main__":
    try:
        root.mainloop()
    except KeyboardInterrupt:
        try:
            parar_backup_automatico(log_ssh)
        except Exception:
            pass
        try:
            root.destroy()
        except Exception:
            pass

C:\Users\Joseffer\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Joseffer\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\Joseffer\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
